MM cell line projj

In [1]:
setwd('~/Downloads/proj/')

library(data.table)
library(edgeR)
library(EDASeq)
library(RUVSeq)
library(RColorBrewer)


MM_cell_line = data.frame(fread("HMCL66_HTSeq_GENE_Counts_v2.txt"))
MM_patients = data.frame(fread("MMRF_CoMMpass_IA13a_E74GTF_HtSeq_Gene_Counts.txt"))

ensembl_intersect = intersect(MM_cell_line$Sample, MM_patients$GENE_ID)

rownames(MM_cell_line) = MM_cell_line$Sample
rownames(MM_patients) = MM_patients$GENE_ID

MM_cell_line = MM_cell_line[ensembl_intersect, -(1:2)]
MM_patients = MM_patients[ensembl_intersect, -1]

MM_combined = cbind(MM_cell_line, MM_patients)

MM_cell_line_cpm = cpm(MM_cell_line)
MM_patients_cpm = cpm(MM_patients)

##pre-processing

# remove low count genes (genes that have 0 counts > 2)

keep <- rowSums(cpm(MM_combined)>1) > 2
table(keep)
MM_combined.filtered <- MM_combined[keep,]

# normalize the samples using upper-quartile/TMM (account for library size)

combined_annot = data.frame(colnames(MM_combined.filtered))
combined_annot$type = "Patient"
combined_annot$type[1:ncol(MM_cell_line)] = "Cell_Line"

set <- newSeqExpressionSet(as.matrix(round(MM_combined.filtered)), phenoData = data.frame(combined_annot, row.names=colnames(MM_combined.filtered)))
set <- betweenLaneNormalization(set, which="upper")


# check for batch effects with RLE plots

x <- as.factor(combined_annot$type)

colors <- brewer.pal(3, "Set2")
#pdf("UQ_plot.pdf")
plotRLE(set, outline=FALSE, ylim=c(-4, 4), col=colors[x])
#dev.off()

# check housekeeping genes by plotting dataset 1 x dataset 2 to see if they're highly correlated; 
# if not then derive empirical controls (see RUVseq package)

#xli = as.character(read.csv('RUVseq_HK_genes.txt'))

#xli = as.character(read.csv('eg.csv')[,2])

#ensembl_intersect2 = intersect(ensembl_intersect, xli)
#print(ensembl_intersect2)
#set1 <- RUVg(set, xli, k=1)
#pData(set1)
#plotRLE(set1, outline=FALSE, ylim=c(-4, 4), col=colors[x])
#plotPCA(set1, col=colors[x], cex=1.2)


# correct for batch effect using RUVg function

##analysis

# correlation between cell lines/patients (code in box)
design <- model.matrix(~x, data=pData(set))
y <- DGEList(counts=counts(set), group=x)
y <- calcNormFactors(y, method="upperquartile")
y <- estimateGLMCommonDisp(y, design)
y <- estimateGLMTagwiseDisp(y, design)
fit <- glmFit(y, design)
lrt <- glmLRT(fit, coef=2)
top <- topTags(lrt, n=nrow(set))$table
empirical <- rownames(set)[which(!(rownames(set) %in% rownames(top)[1:5000]))]

set2 <- RUVg(set, empirical, k=6)
#is.data.frame(set2)

#str(set2)
head(set2)
#write.csv(set2, file = 'set2.csv')
plotRLE(set2, outline=FALSE, ylim=c(-4, 4), col=colors[x])
#plotPCA(set2, col=colors[x], cex=1.2)

MM_cor = cor(normCounts(set2), method="s")

#str(MM_cor)

Loading required package: limma
Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, tab

ERROR: Error: cannot allocate vector of size 355.5 Mb


In [10]:
setwd('~/Downloads/proj/')
xli = scan(file = 'RUVseq_HK_genes.txt', what = character())

str(xli)

library(mygene)

out <- queryMany(xli, scopes="symbol", fields="ensembl.gene", species="human")

print(out)

 chr [1:553] "AATF" "ABL1" "ACAT2" "ACTB" "ACTG1" "ACTN4" "ACVRL1" "ADAM15" ...
Finished
Pass returnall=TRUE to return lists of duplicate or missing query terms.
DataFrame with 553 rows and 5 columns
          query         _id   X_score
    <character> <character> <numeric>
1          AATF       26574  85.72402
2          ABL1          25  86.19428
3         ACAT2          39  90.70537
4          ACTB          60  89.05847
5         ACTG1          71  88.65628
...         ...         ...       ...
549       ZFPL1        7542  89.54935
550      ZNF384      171017 89.070786
551      ZNF592        9640  87.82288
552       ZNF91        7644 99.325096
553      ZNHIT1       10467  90.79195
                                                 ensembl  notfound
                                                  <list> <logical>
1   list(gene = c("ENSG00000276072", "ENSG00000275700"))        NA
2                         list(gene = "ENSG00000097007")        NA
3                         list(gene = 